# Kleber Romon

In [1]:
import requests
import pandas as pd
# en esta parte lo que fue realizado fue el gathering de informaciones referientes as las ordenes y envios
# esta funcion me va a devolver el tipo de logistica según lo que esta identificado en la parte de envios
def logistc_type(type):
        switcher = {
            'fulfillment': "Mercado Envios Full (fulfillment)",
            'drop_off': "Mercado Envios Places (xd_drop_off)",
            'cross_docking': "Mercado Envios Coleta (cross_docking)",
            'self_service':"Mercado Envios Flex (self_service)",

        }
        return switcher.get(type)
# esta funcion va a devolver el cual es el origen del producto segun la logistica definida
def origem(type):
    switcher = {
        'fulfillment': "Depósitos",
    }
    return switcher.get(type, "Vendedor")

# esta funcion hace las conversiones de monedas basado en moneda del item

def conversion_currency(from_currency,to_currency):
    conversions = requests.get('https://api.mercadolibre.com/currency_conversions/search?from='+from_currency+'&to='+to_currency)
    conversions = conversions.json()
    
    return conversions['ratio']
 
# esta funcion va a buscar las informaciones solicitadas para cada una de las ordenes   
    
def Entity_Order(number_order):
    
    Order = requests.get('https://6f008c57-99e0-4a2e-8d80-782a71cf99db.mock.pstmn.io/orders/'+ number +'?caller.id=661750045').json()
    
    itens = Order["order_items"]
    
    descr = ""
    total_amount = 0
    
# este "for" va a validar si hay mas que 1 item por ordem y traer una descripcion de todos los itens de la orden
    for i in range(0,len(itens)):
        
        ID_item = itens[i]['item']['id']

        descr = descr + str(i+1)+". " + str(itens[i]['item']['title']) + ", " + str(itens[i]['item']['variation_attributes'][0]['name']) +' ' +  str(itens[i]['item']['variation_attributes'][0]['value_name'])+'\n'

        total_amount = total_amount + itens[i]['quantity']*itens[i]['full_unit_price']

        currency_id = itens[i]['currency_id']

    taxes = Order["taxes"]

    currency_id_taxes = taxes['currency_id']

    taxes_amount = taxes['amount']
    
# esta funcion va a validar si hay alguna tasa que deba ser cobrada y al mismo tiempo hacer la conversión de cambio
    
    if (taxes_amount != None): 
        if (currency_id_taxes != currency_id):
            ratio = conversion_currency(currency_id_taxes,currency_id)
            taxes_amount = taxes_amount*ratio
    else:
        taxes_amount = 0
    
    ID_shipping = Order["shipping"]["id"]
    
    return Order['id'], ID_item, descr, currency_id, ID_shipping, total_amount, taxes_amount 

# esta funcion va a buscar las informaciones solicitadas para el envio de las ordenes

def Entity_Shipment(ID_shipping):
    
    Shipment = requests.get('https://6f008c57-99e0-4a2e-8d80-782a71cf99db.mock.pstmn.io/shipments/'+str(ID_shipping)+'?caller.id=661750045').json()
    
    ID_Shipment = Shipment["id"]
    
    shipping_cost = Shipment["shipping_option"]['cost']
    
# si hay algun costo para el envio esta función hara la conversión de cambio, de lo contrario la funciona lom dejará como 0
    
    if (shipping_cost!= None):
        if (Shipment["shipping_option"]['currency_id']!= currency_id):
            ratio = conversion_currency(Shipment["shipping_option"]['currency_id'],currency_id)
            taxes_amount = shipping_cost*ratio
    else:
        shipping_cost = 0
        
    status = str(Shipment["status"])+ ", " + Shipment["substatus"]
    
# IF va a validar si el destino es una agencia o "domicilio" tal cual solicitado en el documento enviado

    if Shipment["receiver_address"]["agency"] == None:
        destination = 'Domicílio'
    else:
        destination = 'Agency ID: ' + str(Shipment["receiver_address"]["agency"]["agency_id"]) + ', carrier ID: ' + str(Shipment["receiver_address"]["agency"]["carrier_id"])

#datos del destinatario

    receiver_address = str(Shipment["receiver_address"]["receiver_name"])+", " + str(Shipment["receiver_address"]["street_name"])+", "+ str(Shipment["receiver_address"]["street_number"])+", "+ str(Shipment["receiver_address"]["comment"])+", "+ str(Shipment["receiver_address"]["neighborhood"]["name"]) +", "+ str(Shipment["receiver_address"]["city"]["name"]) +"-"+ str(Shipment["receiver_address"]["state"]["name"])
    
    logistic = Shipment["logistic_type"]
    
    zip_code = Shipment["receiver_address"]["zip_code"]
    
    return ID_Shipment, shipping_cost, status, destination, receiver_address, logistic, zip_code

In [2]:

dataframe = pd.DataFrame()
    
number_order = ["4114988927","4114988960","4114999549"]

# esta función va a agarrar todas las ordenes mencionadas en Number_order y organizalas como en un archivo excel.

for number in number_order:
    
    ID_ordem, ID_item, descr, currency_id, ID_shipping, total_amount, taxes_amount = Entity_Order(number_order)

    ID_Shipment, shipping_cost, status, destination, receiver_address, logistic, zip_code = Entity_Shipment(ID_shipping)

    total_amount_with_shipping = total_amount + taxes_amount + shipping_cost

    df = {'ID ordem': [ID_ordem], 'ID item': [ID_item], 'Descricao do produto': [descr], 'Custo total da ordem': [total_amount_with_shipping], 'Moeda utilizada': [currency_id], 'ID Shipment': [ID_Shipment], 'logística': [logistc_type(logistic)], 'Estado e subestado do envio': [status], 'Origem': [origem(logistic)], 'Destino': [destination], 'Endereço completo': [receiver_address], "CEP / Zipcode": [zip_code] }

    dataframe = pd.concat([dataframe, pd.DataFrame.from_dict(df)], ignore_index = True, axis = 0)


In [3]:
dataframe.to_csv('condicao_negocio.csv', encoding="utf-8-sig")